### Code Hist.

- CODE  
   &ensp; : KIER Data_단순 분리 (전처리 X)  
- DESC  
   &emsp; [수행 작업]  
   &emsp; 1) Date 형식 검사 및 이상치 처리  
   &emsp;&emsp; 1-1) Datetime 유효성 확인  
   &emsp;&emsp; 1-2) Datetime 이상치 제거  
   &emsp; 2) 세대별 사용량 분리  
   &emsp;&emsp; 2-1) HOUSE_ID를 기반으로 적산 사용량 분리  
   &emsp;&emsp; 2-2) 세대별 적산 사용량을 .csv 파일로 저장  
   &emsp;&emsp; 2-3) 세대별 분리된 적산 사용량을 Combine  
- DATE  
   &ensp; 2023-10-12 Created  
   &ensp; 2023-11-21 Code 최신화  
   &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 기존 모듈화된 함수 사용  
   &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) 기존 KIER 변환 관련 코드를 모두 통합  
   &ensp; 2024-01-19 Code 개선  
   &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 공통코드 사용 경로 개선, Raw로 저장  
   &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) HOT (온수 사용량) 전용 Code 작성  
   &ensp; 2024-03-13 Code 개선  
   &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 공통코드 사용 경로 개선, Raw로 저장  
   &ensp; 2024-06-27 Code 개선  
   &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) GAS Domain 추가  

## 01. Code

### 01-01. Init

#### 01-01-01. Init_Module Import

#### Module import

In [1]:
#region Basic_Import
## Basic
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.path.dirname(os.path.abspath('__file__'))
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))

import warnings
warnings.filterwarnings('ignore')

import numpy as np, pandas as pd
from pandas import DataFrame, Series

import math, random

## Datetime
import time
import datetime as dt
from datetime import datetime, date, timedelta

import glob
from glob import glob
import requests
import json

## 시각화
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

# K-Means 알고리즘
from sklearn.cluster import KMeans, MiniBatchKMeans

# CLustering 알고리즘의 성능 평가 측도
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, silhouette_score, rand_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics.cluster import contingency_matrix

## 정규화
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import metrics

import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

from tqdm.notebook import tqdm
import tqdm

## Init.
pd.options.display.float_format = '{:.10f}'.format
#endregion Basic_Import

In [2]:
## Import_DL
str_tar = "tf"
## For Torch
if str_tar == "torch":
    import torch
    import torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf
    import tensorflow_addons as tfa
    from keras.callbacks import EarlyStopping, ModelCheckpoint
    from keras.models import Sequential, load_model
    from keras_flops import get_flops
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Tensorflow Imported


In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU만 사용하도록 제한
  try:
    print('Using GPU')
    tf.config.set_visible_devices(gpus[0], 'GPU')
  except RuntimeError as e:
    # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
    print(e)
else : 
  print('Using CPU')

Using GPU


In [4]:
## Import_Local
from Src_Dev_Common import DEV_Common_Data_Datetime as com_date
# from Src_Dev_Common import DEV_KASI_Holiday as com_Holi
# from Src_Dev_Common import DEV_Common_Data_Analysis as com_Analysis
# from Src_Dev_Common import DEV_Common_Data_Preprocessing as com_Prep
# from Src_Dev_Common import DEV_Common_Model as com_Model
# from Src_Dev_Common import DEV_Common_Data_Visualization as com_Visual
# from Src_Dev_Common import DEV_KMA_Weather_ASOS as com_ASOS
# from Src_Dev_Common import DEV_KDHC_Usage as com_KDHC
# from Src_Dev_Common import DEV_KIER_Usage as com_KIER

### 01-01-02. Config (Directory, Params)

In [5]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = "1"
random.seed(SEED)

In [6]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y, str_now_m, str_now_d = pd.datetime.now().year, pd.datetime.now().month, pd.datetime.now().day
str_now_hr, str_now_min = pd.datetime.now().hour, pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2024-07-02 23:56:03.430822
2024 / 7 / 2
23 : 56


In [7]:
## Dict_Domain
dict_domain = {0:"ELEC", 1:"HEAT", 2:"WATER", 3:"HOT_HEAT", 4:"HOT_FLOW", 5:"GAS"} ## GAS는 사용하지 않음.
int_domain = 5
str_domain = str(dict_domain[int_domain])

dict_col_accu = {0 : "ACTUAL_ACCU_EFF" ## ELEC
                 , 1 : "ACCU_HEAT" ## HEAT
                 , 2 : "ACCU_FLOW" ## WATER
                 , 3 : "ACCU_HEAT" ## HOT 열량
                 , 4 : "ACCU_FLOW" ## HOT 유량
                 , 5 : "ACCU_FLOW" ## GAS
                 }
str_col_accu = str(str_domain + "_" + str(dict_col_accu[int_domain]))

dict_col_inst = {0 : "INST_EFF" ## ELEC_INST_EFF
                , 1 : "INST_HEAT" ## HEAT_INST_HEAT
                , 2 : "INST_FLOW" ## WATER_INST_FLOW
                , 3 : "INST" ## HOT_HEAT_INST
                , 4 : "INST" ## HOT_FLOW_INST
                , 5 : "INST_FLOW" ## GAS_INST_FLOW
                } 
str_col_inst = str(str_domain + "_" + str(dict_col_inst[int_domain]))

## Directory Root
str_dirData = "../data/data_Energy_KIER/"
str_dir_raw = '../data/data_Energy_KIER/KIER_0_Raw/'
str_dirName_bld = '../data/data_Energy_KIER/KIER_1_BLD/'
str_dirName_h = '../data/data_Energy_KIER/KIER_2_H_' + str_domain + '/'

## File
str_fileRaw = str('KIER_RAW_' + str_domain + '_2024-06-07.csv')
str_fileRaw_hList = str('KIER_hList_Common.csv') 

print(str(os.listdir(str_dirData)) + "\n")
print(os.listdir(str_dirName_h))

['.ipynb_checkpoints', 'BS_CONFIGURATION_202309251452.csv', 'DATE_1M_2023-10-20.csv', 'KIER 전처리 현황_2024-06-25.xlsx', 'KIER_0_Raw', 'KIER_1_BLD', 'KIER_2_H_ELEC', 'KIER_2_H_GAS', 'KIER_2_H_HEAT', 'KIER_2_H_HOT_FLOW', 'KIER_2_H_HOT_HEAT', 'KIER_2_H_WATER', 'KIER_ASOS_WEATHER_DAILY_202309251521.csv', 'KIER_ASOS_WEATHER_HOUR_202309251521.csv', 'KIER_DATA_OLD', 'KIER_ETC', 'KIER_hList_Comparison_2024-06-26.xlsx', 'KIER_List_Table_Column_2023-09-25.xlsx', 'KIER_Query_2023-09-25.txt', 'KMA_ASOS_119_2010_2023_1st_to CSV.csv', '[IITP] 데이터 테이블 정리 (공유 원본).docx']

['KIER_GAS_561-1-1_ACCU_01_Raw.csv', 'KIER_GAS_561-1-1_INST_01_10min.csv', 'KIER_GAS_561-1-2_ACCU_01_Raw.csv', 'KIER_GAS_561-1-2_INST_01_10min.csv', 'KIER_GAS_561-1-3_ACCU_01_Raw.csv', 'KIER_GAS_561-1-3_INST_01_10min.csv', 'KIER_GAS_561-1-4_ACCU_01_Raw.csv', 'KIER_GAS_561-1-4_INST_01_10min.csv', 'KIER_GAS_561-10-1_ACCU_01_Raw.csv', 'KIER_GAS_561-10-1_INST_01_10min.csv', 'KIER_GAS_561-10-2_ACCU_01_Raw.csv', 'KIER_GAS_561-10-2_INST_01_10mi

In [8]:
## "KIER_01-01_Data_hList.ipynb"로부터 만들어진 Bld/F/H List
df_kier_hList = pd.read_csv(str_dir_raw + str_fileRaw_hList, index_col = 0)
list_h_id = df_kier_hList['HOUSE_ID'].drop_duplicates()
print(df_kier_hList.shape, " /// ", df_kier_hList.columns)
df_kier_hList

(348, 5)  ///  Index(['HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO_INT', 'HOUSE_ID',
       'HOUSE_ID_HO'],
      dtype='object')


,HOUSE_ID_DONG,HOUSE_ID_HO_PRE,HOUSE_ID_HO_INT,HOUSE_ID,HOUSE_ID_HO
1,561,1,1,561-1-1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=
2,561,1,2,561-1-2,ODBEOTE0QzZCMjg5RUMzNTRBQjM1NjE0RDNBNDE0NzU=
3,561,1,3,561-1-3,QTUwRUJDQTlBMkZENUQyNjg0RDRGREVFMTNGNDI3NUY=
4,561,1,4,561-1-4,RkRCQzY4OTlCMTRDQjgzNUZBREE3Njc2MkM2MUM2QjI=
5,561,2,1,561-2-1,MTQxNkQ1RDAzOTE5NTM5RUM3MzlDQkY3RTI2NERCQTM=
...,...,...,...,...,...
344,563,23,2,563-23-2,QkJCQjQwOThFNjgwNjUwQzJGRUJFNDlCNTREOEUxNDU=
345,563,23,3,563-23-3,QUJENzFFMEFDRjM4MjY3NTRDQzJCRjdDNUQ3ODA0N0Y=
346,563,23,4,563-23-4,RUVENzZDNTc4MUY4NDg5MUREQjZEOUM1M0Q2RjYzMDE=
347,563,24,1,563-24-1,NEVENDc1OTFFNDBBRjBDQzkzMTlEMDgzMkMzMkZFQTg=


## 01-02. Data Load (df_raw)

### 01-02-01. KIER (Energy Usage)

In [9]:
## "METER_DATE" Cleansing이 완료된 경우
str_fileCleansed = str('KIER_' + str_domain + '_ACCU_10MIN.csv')
df_cleansed = pd.read_csv(str_dirName_h + str_fileCleansed, index_col = 0)
df_cleansed = com_date.create_col_datetime(df_cleansed, 'METER_DATE', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE')
df_cleansed

,YEAR,MONTH,DAY,HOUR,MINUTE,GAS_ACCU_FLOW_561-1-1,GAS_ACCU_FLOW_561-1-2,GAS_ACCU_FLOW_561-1-3,GAS_ACCU_FLOW_561-1-4,GAS_ACCU_FLOW_561-2-1,...,GAS_ACCU_FLOW_563-23-1,GAS_ACCU_FLOW_563-23-2,GAS_ACCU_FLOW_563-23-3,GAS_ACCU_FLOW_563-23-4,GAS_ACCU_FLOW_563-24-1,GAS_ACCU_FLOW_563-24-2,MEAN_OF_ACCU,SUM_OF_ACCU,None,METER_DATE
0,2022,7,17,23,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000000000,0,2022-07-17 23:00:00
1,2022,7,17,23,10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000000000,0,2022-07-17 23:10:00
2,2022,7,17,23,20,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000000000,0,2022-07-17 23:20:00
3,2022,7,17,23,30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000000000,0,2022-07-17 23:30:00
4,2022,7,17,23,40,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000000000,0,2022-07-17 23:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99166,2024,6,5,14,40,NaN,0.1300000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,43.2154545455,475.3700000000,0,2024-06-05 14:40:00
99167,2024,6,5,14,50,NaN,0.1300000000,113.3200000000,NaN,0.1100000000,...,NaN,NaN,NaN,NaN,NaN,NaN,43.1694871795,1683.6100000000,0,2024-06-05 14:50:00
99168,2024,6,5,15,0,NaN,0.1300000000,113.3200000000,NaN,0.1100000000,...,NaN,NaN,NaN,NaN,NaN,NaN,44.2972500000,1771.8900000000,0,2024-06-05 15:00:00
99169,2024,6,5,15,10,0.1300000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,58.6014925373,3926.3000000000,0,2024-06-05 15:10:00


### 순시사용량

In [10]:
cnt_minus = 0
for h_id in list_h_id:
    print(h_id)
    
    str_col_accu_h, str_col_inst_h = str_col_accu + "_" + h_id, str_col_inst + "_" + h_id
    df_h_tmp = df_cleansed[['METER_DATE', str_col_accu_h]]

    df_h_tmp[str_col_inst_h] = 0
    for i in range(0, len(df_h_tmp) - 1):
        usage_inst = df_h_tmp[str_col_accu_h].iloc[i + 1] - df_h_tmp[str_col_accu_h].iloc[i]

        ## 순시사용량이 음수인 경우, 해당 값은 nan으로 입력
        if usage_inst < 0 : 
            df_h_tmp[str_col_inst_h].iloc[i] = np.nan
            cnt_minus = cnt_minus + 1
        else : df_h_tmp[str_col_inst_h].iloc[i] = usage_inst
        
    df_h_tmp = df_h_tmp.reset_index()[['METER_DATE', str_col_inst_h]]
    
    str_file = 'KIER_' + str_domain + '_' + str(h_id) + '_INST_01_10min.csv'
    df_h_tmp.to_csv(str_dirName_h + str_file)

print(cnt_minus)
df_h_tmp

561-1-1
561-1-2
561-1-3
561-1-4
561-2-1
561-2-2
561-2-3
561-2-4
561-3-1
561-3-2
561-3-3
561-3-4
561-4-1
561-4-2
561-4-3
561-4-4
561-5-1
561-5-2
561-5-3
561-5-4
561-6-1
561-6-2
561-6-3
561-6-4
561-7-1
561-7-2
561-7-3
561-7-4
561-8-1
561-8-2
561-8-3
561-8-4
561-9-1
561-9-2
561-9-3
561-9-4
561-10-1
561-10-2
561-10-3
561-10-4
561-11-1
561-11-2
561-11-3
561-11-4
561-12-1
561-12-2
561-12-3
561-12-4
561-13-1
561-13-2
561-13-3
561-13-4
561-14-1
561-14-2
561-14-3
561-14-4
561-15-1
561-15-2
561-15-3
561-15-4
561-16-1
561-16-2
561-16-3
561-16-4
561-17-1
561-17-2
561-17-3
561-17-4
561-18-1
561-18-2
561-18-3
561-18-4
562-1-1
562-1-2
562-1-3
562-1-4
562-1-5
562-1-6
562-2-1
562-2-2
562-2-3
562-2-4
562-2-5
562-2-6
562-3-1
562-3-2
562-3-3
562-3-4
562-3-5
562-3-6
562-4-1
562-4-2
562-4-3
562-4-4
562-4-5
562-4-6
562-5-1
562-5-2
562-5-3
562-5-4
562-5-5
562-5-6
562-6-1
562-6-2
562-6-3
562-6-4
562-6-5
562-6-6
562-7-1
562-7-2
562-7-3
562-7-4
562-7-5
562-7-6
562-8-1
562-8-2
562-8-3
562-8-4
562-8-5
562-8-6
562-

,METER_DATE,GAS_INST_FLOW_563-24-2
0,2022-07-17 23:00:00,NaN
1,2022-07-17 23:10:00,NaN
2,2022-07-17 23:20:00,NaN
3,2022-07-17 23:30:00,NaN
4,2022-07-17 23:40:00,NaN
...,...,...
99166,2024-06-05 14:40:00,NaN
99167,2024-06-05 14:50:00,NaN
99168,2024-06-05 15:00:00,NaN
99169,2024-06-05 15:10:00,NaN


#### 모든 호실의 사용량을 변수화하여 한 데이터셋에 Combine

In [11]:
int_cnt = 0
for h_id in list_h_id:
    print(h_id)
    str_col_inst_h = str_col_inst + "_" + h_id

    str_file = 'KIER_' + str_domain + '_' + str(h_id) + '_INST_01_10min.csv'
    df_h_tmp = pd.read_csv(str_dirName_h + str_file, index_col = 0).reset_index()[['METER_DATE', str_col_inst_h]]

    df_h_tmp['METER_DATE'] = pd.to_datetime(df_h_tmp['METER_DATE'])

    if int_cnt == 0 : df_kier_h_Combined = df_h_tmp.copy()
    else : df_kier_h_Combined = pd.merge(df_kier_h_Combined, df_h_tmp, how = 'left', on = ['METER_DATE'])
    
    int_cnt = int_cnt + 1

## 범위 외 이상 데이터를 모두 제외
start_date = pd.to_datetime('2022-07-17 23:00:00') ## 시작 날짜
end_date = pd.to_datetime('2024-06-05 15:20:00') ## 마지막 날짜

df_dt = pd.DataFrame()
df_dt = com_date.create_df_dt(df_dt, 'METER_DATE', start_date, end_date, '10min')
df_kier_h_Combined = pd.merge(df_dt, df_kier_h_Combined, how = 'left', on = ['METER_DATE'])

## Cleansed Data를 저장
str_file = 'KIER_' + str_domain + '_INST_01_10min.csv'
df_kier_h_Combined.to_csv(str_dirName_h + str_file)
df_kier_h_Combined

561-1-1
561-1-2
561-1-3
561-1-4
561-2-1
561-2-2
561-2-3
561-2-4
561-3-1
561-3-2
561-3-3
561-3-4
561-4-1
561-4-2
561-4-3
561-4-4
561-5-1
561-5-2
561-5-3
561-5-4
561-6-1
561-6-2
561-6-3
561-6-4
561-7-1
561-7-2
561-7-3
561-7-4
561-8-1
561-8-2
561-8-3
561-8-4
561-9-1
561-9-2
561-9-3
561-9-4
561-10-1
561-10-2
561-10-3
561-10-4
561-11-1
561-11-2
561-11-3
561-11-4
561-12-1
561-12-2
561-12-3
561-12-4
561-13-1
561-13-2
561-13-3
561-13-4
561-14-1
561-14-2
561-14-3
561-14-4
561-15-1
561-15-2
561-15-3
561-15-4
561-16-1
561-16-2
561-16-3
561-16-4
561-17-1
561-17-2
561-17-3
561-17-4
561-18-1
561-18-2
561-18-3
561-18-4
562-1-1
562-1-2
562-1-3
562-1-4
562-1-5
562-1-6
562-2-1
562-2-2
562-2-3
562-2-4
562-2-5
562-2-6
562-3-1
562-3-2
562-3-3
562-3-4
562-3-5
562-3-6
562-4-1
562-4-2
562-4-3
562-4-4
562-4-5
562-4-6
562-5-1
562-5-2
562-5-3
562-5-4
562-5-5
562-5-6
562-6-1
562-6-2
562-6-3
562-6-4
562-6-5
562-6-6
562-7-1
562-7-2
562-7-3
562-7-4
562-7-5
562-7-6
562-8-1
562-8-2
562-8-3
562-8-4
562-8-5
562-8-6
562-

,METER_DATE,day_of_the_week,YEAR,MONTH,DAY,HOUR,MINUTE,code_day_of_the_week,GAS_INST_FLOW_561-1-1,GAS_INST_FLOW_561-1-2,...,GAS_INST_FLOW_563-22-3,GAS_INST_FLOW_563-22-4,GAS_INST_FLOW_563-22-5,GAS_INST_FLOW_563-22-6,GAS_INST_FLOW_563-23-1,GAS_INST_FLOW_563-23-2,GAS_INST_FLOW_563-23-3,GAS_INST_FLOW_563-23-4,GAS_INST_FLOW_563-24-1,GAS_INST_FLOW_563-24-2
0,2022-07-17 23:00:00,Sunday,2022,7,17,23,0,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-07-17 23:10:00,Sunday,2022,7,17,23,10,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-07-17 23:20:00,Sunday,2022,7,17,23,20,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-07-17 23:30:00,Sunday,2022,7,17,23,30,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-07-17 23:40:00,Sunday,2022,7,17,23,40,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99166,2024-06-05 14:40:00,Wednesday,2024,6,5,14,40,2,NaN,0.0000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99167,2024-06-05 14:50:00,Wednesday,2024,6,5,14,50,2,NaN,0.0000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99168,2024-06-05 15:00:00,Wednesday,2024,6,5,15,0,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99169,2024-06-05 15:10:00,Wednesday,2024,6,5,15,10,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
list_col = df_kier_h_Combined.columns[1:]
df_dt_period = df_kier_h_Combined[['METER_DATE']]

df_kier_Calc = df_kier_h_Combined[list_col]
df_kier_extract = df_kier_h_Combined

## 순시 사용량의 합계 및 평균 Column을 추가
df_kier_extract['MEAN_OF_INST'] = df_kier_Calc.mean(axis = 1)
df_kier_extract['SUM_OF_INST'] = df_kier_Calc.sum(axis = 1)

str_file = 'KIER_' + str_domain + '_INST_01_10min.csv'
df_kier_extract.to_csv(str_dirName_h + str_file)

df_kier_extract

,METER_DATE,day_of_the_week,YEAR,MONTH,DAY,HOUR,MINUTE,code_day_of_the_week,GAS_INST_FLOW_561-1-1,GAS_INST_FLOW_561-1-2,...,GAS_INST_FLOW_563-22-5,GAS_INST_FLOW_563-22-6,GAS_INST_FLOW_563-23-1,GAS_INST_FLOW_563-23-2,GAS_INST_FLOW_563-23-3,GAS_INST_FLOW_563-23-4,GAS_INST_FLOW_563-24-1,GAS_INST_FLOW_563-24-2,MEAN_OF_INST,SUM_OF_INST
0,2022-07-17 23:00:00,Sunday,2022,7,17,23,0,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,345.8333333333,2075.0000000000
1,2022-07-17 23:10:00,Sunday,2022,7,17,23,10,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,347.5000000000,2085.0000000000
2,2022-07-17 23:20:00,Sunday,2022,7,17,23,20,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,349.1666666667,2095.0000000000
3,2022-07-17 23:30:00,Sunday,2022,7,17,23,30,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,350.8333333333,2105.0000000000
4,2022-07-17 23:40:00,Sunday,2022,7,17,23,40,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,352.5000000000,2115.0000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99166,2024-06-05 14:40:00,Wednesday,2024,6,5,14,40,2,NaN,0.0000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,174.2500000000,2091.0000000000
99167,2024-06-05 14:50:00,Wednesday,2024,6,5,14,50,2,NaN,0.0000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.7500000000,2101.0000000000
99168,2024-06-05 15:00:00,Wednesday,2024,6,5,15,0,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,342.0000000000,2052.0000000000
99169,2024-06-05 15:10:00,Wednesday,2024,6,5,15,10,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.8717948718,2062.0000000000


In [13]:
str_file = 'KIER_' + str_domain + '_INST_01_10min.csv'
df_kier_tmp = pd.read_csv(str_dirName_h + str_file, index_col = 0)
df_kier_tmp

,METER_DATE,day_of_the_week,YEAR,MONTH,DAY,HOUR,MINUTE,code_day_of_the_week,GAS_INST_FLOW_561-1-1,GAS_INST_FLOW_561-1-2,...,GAS_INST_FLOW_563-22-5,GAS_INST_FLOW_563-22-6,GAS_INST_FLOW_563-23-1,GAS_INST_FLOW_563-23-2,GAS_INST_FLOW_563-23-3,GAS_INST_FLOW_563-23-4,GAS_INST_FLOW_563-24-1,GAS_INST_FLOW_563-24-2,MEAN_OF_INST,SUM_OF_INST
0,2022-07-17 23:00:00,Sunday,2022,7,17,23,0,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,345.8333333333,2075.0000000000
1,2022-07-17 23:10:00,Sunday,2022,7,17,23,10,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,347.5000000000,2085.0000000000
2,2022-07-17 23:20:00,Sunday,2022,7,17,23,20,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,349.1666666667,2095.0000000000
3,2022-07-17 23:30:00,Sunday,2022,7,17,23,30,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,350.8333333333,2105.0000000000
4,2022-07-17 23:40:00,Sunday,2022,7,17,23,40,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,352.5000000000,2115.0000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99166,2024-06-05 14:40:00,Wednesday,2024,6,5,14,40,2,NaN,0.0000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,174.2500000000,2091.0000000000
99167,2024-06-05 14:50:00,Wednesday,2024,6,5,14,50,2,NaN,0.0000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.7500000000,2101.0000000000
99168,2024-06-05 15:00:00,Wednesday,2024,6,5,15,0,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,342.0000000000,2052.0000000000
99169,2024-06-05 15:10:00,Wednesday,2024,6,5,15,10,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.8717948718,2062.0000000000
